# Session 2 Scraping
- Why is scrapping useful?
- Basics of How Browser Works
- How to clean from json, 
- Wikipeadia getting back to Phylisophy
- Twitter API
- Reading Data from DataBase db

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import re
import bs4
import requests

<p style='font-size:20px'>
Suppose that we wish to build a system that warns us when stock prices starts to go up or go down. The first thing we want to do is to scrape a website to get the data. <br>
Specifically, get data from website https://finance.yahoo.com/most-active
</p>

In [ ]:
url = 'https://finance.yahoo.com/most-active'
res = requests.get(url=url)
soup = bs4.BeautifulSoup(res.text, "html.parser")

In [ ]:
res

In [ ]:
with open('page.html','a') as f:
    f.write(soup.prettify())

In [ ]:
# print(soup.prettify())
tabl = soup.select_one('#scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table') ## Copied using Chrome Extension #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
# #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table

thead = tabl.find('thead')
tbody = tabl.find('tbody')

head_lst = [hd.text for hd in thead.find_all('th')]

tab_lst = list()
for tr in tbody.find_all('tr'):
    tr_lst = list()
    for td in tr.find_all('td'):
        tr_lst.append(td.text)
    tab_lst.append(tr_lst)

In [ ]:
df_stock = pd.DataFrame(tab_lst, columns=head_lst)
df_stock

In [ ]:
df_stock['Change'] = df_stock['Change'].astype(float)
df_stock['% Change'] = df_stock['% Change'].str.rstrip('%').astype('float') / 100.0
df_stock['Volume'] = df_stock['Volume'].str.rstrip('M').astype(float)
df_stock['Avg Vol (3 month)'] = df_stock['Avg Vol (3 month)'].str.rstrip('M').astype(float)
df_stock['Price (Intraday)'] = df_stock['Price (Intraday)'].astype(float)

In [ ]:
df_stock.describe()

In [ ]:
df_stock[df_stock['Change'] == df_stock['Change'].max()]

## Exercise 1: Scrape the Table from Cancer Statistics
Go to the site https://www.cdc.gov/cancer/kinds.htm
Scrape the explaination for each cancer in the first cards, store it in a dictionary.

In [ ]:
url = 'https://www.cdc.gov/cancer/kinds.htm'
res = requests.get(url=url)
soup = bs4.BeautifulSoup(res.text, "html.parser")

## Dynamic Loading and Writing CSV files

In [ ]:
df_stock

In [ ]:
df_stock.to_csv('stocks.csv', index=False)

In [ ]:
import csv

In [ ]:
f = open('stocks.csv')

In [ ]:
reader = csv.reader(f)

In [ ]:
header = next(reader)

In [ ]:
rows = []
for rw in reader:
    rows.append(rw)

In [ ]:
pd.DataFrame(rows, columns=header)

In [ ]:
f.close()

In [ ]:
rows = []

In [ ]:
with open('stocks.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    rows = []
    for rw in reader:
        rows.append(rw)

In [ ]:
pd.DataFrame(rows, columns=header)

In [ ]:
def scrape_yahoo():
    url = 'https://finance.yahoo.com/most-active'
    res = requests.get(url=url)
    soup = bs4.BeautifulSoup(res.text, "html.parser")

    tabl = soup.select_one('#scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table') ## Copied using Chrome Extension #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
    # #scr-res-table > div.Ovx\(a\).Ovx\(h\)--print.Ovy\(h\).W\(100\%\) > table
    
    thead = tabl.find('thead')
    tbody = tabl.find('tbody')
    
    head_lst = [hd.text for hd in thead.find_all('th')]
    
    tab_lst = list()
    for tr in tbody.find_all('tr'):
        tr_lst = list()
        for td in tr.find_all('td'):
            tr_lst.append(td.text)
        tab_lst.append(tr_lst)

    return pd.DataFrame(tab_lst, columns=head_lst)

In [ ]:
df_stock = scrape_yahoo()

In [ ]:
stockdic = df_stock.transpose().to_dict()

In [ ]:
header = df_stock.columns

In [ ]:
with open('stocks_appd.csv','a+') as fa:
    csvwriter = csv.DictWriter(fa, fieldnames=header)
    csvwriter.writeheader()
    for key, dic_i in stockdic.items():
        csvwriter.writerow(dic_i)

## Wikipedia Game

### 3. Snscrape
Snscrape provides many useful attributes available through snscrape Twitter object [(Beck, 2020)](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af). You might need some of the above attributes when scraping data for your group projects.
* url: Permalink pointing to tweet location
* date: The date on which the tweet was created
* content: The text content of the tweet
* id: The id of the tweet
* user: User object containing the following data: username, displayname, id, description, descriptionUrls, verified, created, followersCount, friendsCount, statusesCount, favouritesCount, listedCount, location, protected, linkUrl, profileImageUrl, profileBannerUrl
* replyCount: The count of replies
* retweetCount: The count of retweets
* likeCount: The count of likes
* quoteCount: The count of users that quoted the tweet and replied
* conversationId: Appears to be the same as tweet id
* lang: Machine generated, assumed language of the tweet
* source: Where tweet was posted from (e.g., iPhone, Andriod, etc.)
* retweetedTweet: The id of the original tweet (if it is a retweet)
* mentionedUsers: The user objects of any mentioned user in the tweet

In [ ]:
import snscrape.modules.twitter as sntwitter
from tqdm import tqdm

In [ ]:
tweets_list2 = []

# use TwitterSearchScraper to scrape data and append tweets to list
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('let\'s go duke since:2021-11-12 until:2021-11-13').get_items()):
    tweets_list2.append([tweet.date, tweet.content, tweet.user.location])

In [ ]:
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Date', 'Content', 'User\'s location'])
tweets_df2

### Exercise 3: Scrape twitter related to cancer